In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import os
from os import listdir

In [ ]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]
dfTuesday=pd.DataFrame()
for idx,file in enumerate(find_csv_filenames('Netflow/CIC-IDS2017/pcaps/Benign')):
    print(file,idx)
    if dfTuesday.empty:
        df=pd.read_csv('Netflow/CIC-IDS2017/pcaps/Benign/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[0 for i in range(len(df))]})],axis=1)
        dfMonday=df
    else:
        df=pd.read_csv('Netflow/CIC-IDS2017/pcaps/Benign/'+file)
        df=pd.concat([df,pd.DataFrame({"label":[0 for i in range(len(df))]})],axis=1)
        dfMonday=pd.concat([dfMonday,df])
dfMonday = dfMonday.sort_values("ts")
pd.set_option('display.max_columns', None)

In [ ]:
print(dfMonday.dtos.value_counts())
print(dfMonday.pr.value_counts())
print(dfMonday.head())
print(dfMonday.columns)
print(len(dfMonday.columns))

In [ ]:
netflow = pd.concat([dfMonday])

In [ ]:
print(netflow.label.unique())
print(netflow.shape)

In [ ]:
conn_order = ["ts", "uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p", "proto", "service", "duration", "orig_bytes", "resp_bytes", "conn_state", "local_orig", "local_resp", "missed_bytes", "history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes",	"tunnel_parents"]

f=open("Netflow/CIC-IDS2017/pcaps/Tuesday/conn.log",'r')
lines = f.readlines()
data = []
for line in lines[8:-1]:
    details = line.split('	')
    details = [x.strip() for x in details]
    structure = {key: value for key, value in zip(conn_order, details)}
    data.append(structure)

df_temp = pd.DataFrame(data, columns=data[0].keys())
anomaly = [1] * len(df_temp.index)
df_temp['anomaly'] = anomaly
#df_temp['ts_new'] = pd.to_datetime(df_temp['ts'], utc=True)
df_temp=df_temp.sort_values('ts')
df_temp['ts'] = df_temp['ts'].apply(lambda x: int(x.split('.')[0]))
df_temp['ts'] = df_temp['ts'].apply(lambda x: str(datetime.fromtimestamp(x)))


In [ ]:
zeek = pd.concat([df_temp])
zeek.columns
print(len(zeek.columns))

In [ ]:
print(zeek.columns)
print(netflow.columns)
#print(netflow.dir.value_counts())

In [ ]:
print(zeek.shape)
print(netflow.shape)

In [ ]:
zeek.duration=pd.to_numeric(zeek.duration,errors='coerce')
zeek=zeek.dropna()
#zeek.ts=zeek.ts.astype(float)#+zeek.duration
#zeek.ts=zeek.ts-zeek.ts.iloc[0]
#zeek=zeek.sort_values('ts')
#print(zeek.ts)

netflow=netflow.dropna()
netflow.dp=netflow.dp.astype(int)
from datetime import datetime
from time import mktime
#netflow['ts'] =netflow['ts'].apply(lambda x: mktime(datetime.strptime(x,'%Y-%m-%d %H:%M:%S').timetuple()))
#netflow.ts=netflow.ts#+netflow.td.astype(float)
#netflow.ts=netflow.ts-netflow.ts.iloc[0]
#netflow=netflow.sort_values('ts')
#print(netflow.ts)

In [ ]:
print(netflow)
print(zeek)

In [ ]:
netflow.sa=netflow.sa.astype(str)
netflow.da=netflow.da.astype(str)
netflow.sp=netflow.sp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow.ipkt=netflow.ipkt.astype(int)
netflow.opkt=netflow.opkt.astype(int)

zeek['id.orig_h']=zeek['id.orig_h'].astype(str)
zeek['id.resp_h']=zeek['id.resp_h'].astype(str)
zeek['id.orig_p']=zeek['id.orig_p'].astype(int)
zeek['id.resp_p']=zeek['id.resp_p'].astype(int)
zeek['orig_pkts']=zeek['orig_pkts'].astype(int)
zeek['resp_pkts']=zeek['resp_pkts'].astype(int)
netflow.sp=netflow.sp.astype(int)
netflow.dp=netflow.dp.astype(int)
netflow_to_zeek={"sa": "id.orig_h", "da": "id.resp_h","sp":"id.orig_p","dp":"id.resp_p"}
netflow=netflow.rename(columns=netflow_to_zeek)
common_cols = ['id.orig_h','id.resp_h','id.orig_p','id.resp_p']#,'orig_pkts','resp_pkts'] 
#netflow=netflow.drop_duplicates(subset=common_cols,keep='last')
zeek=zeek.drop_duplicates(subset=common_cols,keep='last')
df12 = pd.merge(netflow, zeek, on=common_cols, how='left')     #extract common rows with merge
#df12=df12.dropna()
df2 = zeek[~zeek['uid'].isin(df12['uid'])]
#print(df12.columns)
df3 = netflow[~netflow['ts'].isin(df12['ts_x'])]
print(df12.shape)
print(df2.shape)
print(df3.shape)
'''print(df2.iloc[0])
#print(df3.iloc[0])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df2[['ts']+common_cols])
print(df3[['ts']+common_cols])'''
zeek_to_netflow={v: k for k, v in netflow_to_zeek.items()}
df12=df12.rename(columns=zeek_to_netflow)
df12=df12.drop(['ts_y'],axis=1)
df12=df12.rename(columns={'ts_x':'ts'})
netflow=netflow.rename(columns=zeek_to_netflow)

In [ ]:
df12[netflow.columns.to_list()+['history']]

In [ ]:
print(df12.label.value_counts())
print(df12.anomaly.value_counts())
print(df12.history.info())

In [ ]:
df12.to_csv('Merged Zeek-Netflow/cic_monday.csv',index=False)

In [ ]:
print(len(df12))
print(df12['history'].isna().sum())
print(df12.isna())
print(df12.label.unique())
print(df12.sa.value_counts())